# Skip-gram word2vec

TensorFlow implementation of word2vec algorithm using the skip-gram architecture.

In [1]:
import time

import numpy as np
import tensorflow as tf

print(tf.__version__)

2024-03-06 22:17:21.519812: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2.11.0


Load the [text8 dataset](http://mattmahoney.net/dc/textdata.html), a file of cleaned up Wikipedia articles from Matt Mahoney.

In [2]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
from tqdm import tqdm
import zipfile

dataset_folder_path = 'data'
dataset_filename = 'text8.zip'
dataset_name = 'Text8 Dataset'

class DLProgress(tqdm):
    last_block = 0

    def hook(self, block_num=1, block_size=1, total_size=None):
        self.total = total_size
        self.update((block_num - self.last_block) * block_size)
        self.last_block = block_num

if not isfile(dataset_filename):
    with DLProgress(unit='B', unit_scale=True, miniters=1, desc=dataset_name) as pbar:
        urlretrieve(
            'http://mattmahoney.net/dc/text8.zip',
            dataset_filename,
            pbar.hook)

if not isdir(dataset_folder_path):
    with zipfile.ZipFile(dataset_filename) as zip_ref:
        zip_ref.extractall(dataset_folder_path)
        
with open('data/text8') as f:
    text = f.read()

In [3]:
print(text[:1000])

 anarchism originated as a term of abuse first used against early working class radicals including the diggers of the english revolution and the sans culottes of the french revolution whilst the term is still used in a pejorative way to describe any act that used violent means to destroy the organization of society it has also been taken up as a positive label by self defined anarchists the word anarchism is derived from the greek without archons ruler chief king anarchism as a political philosophy is the belief that rulers are unnecessary and should be abolished although there are differing interpretations of what this means anarchism also refers to related social movements that advocate the elimination of authoritarian institutions particularly the state the word anarchy as most anarchists use it does not imply chaos nihilism or anomie but rather a harmonious anti authoritarian society in place of what are regarded as authoritarian political structures and coercive economic instituti

#### Word2Vec parameters

In [4]:
# Total number of different words in the vocabulary.
max_vocabulary_size = 50000
min_occurrence = 10  # Remove all words that does not appears at least n times.

## Preprocessing

In [5]:
def preprocess(text):
    words = text.lower().split()
    return words

In [6]:
tokens = preprocess(text)
print(tokens[:30])

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against', 'early', 'working', 'class', 'radicals', 'including', 'the', 'diggers', 'of', 'the', 'english', 'revolution', 'and', 'the', 'sans', 'culottes', 'of', 'the', 'french', 'revolution', 'whilst']


In [7]:
print("Total tokens: {}".format(len(tokens)))
print("Unique tokens: {}".format(len(set(tokens))))

Total tokens: 17005207
Unique tokens: 253854


In [8]:
from collections import Counter
# Build the dictionary and replace rare words with UNK token.
count = [('UNK', -1)]
count.extend(Counter(tokens).most_common(max_vocabulary_size - 1))

# Remove samples with less than 'min_occurrence' occurrences.
for i in range(len(count) - 1, -1, -1):
    if count[i][1] < min_occurrence:
        count.pop(i)
    else:
        # The collection is ordered, so stop when 'min_occurrence' is reached.
        break
# Compute the vocabulary size.
vocabulary_size = len(count)
print(vocabulary_size)

47135


In [9]:
# Assign an id to each word.
word2id = dict()
for i, (word, _) in enumerate(count):
    word2id[word] = i

data = list()
unk_count = 0
for word in tokens:
    # Retrieve a word id, or assign it index 0 ('UNK') if not in dictionary.
    index = word2id.get(word, 0)
    if index == 0:
        unk_count += 1
    data.append(index)
count[0] = ('UNK', unk_count)
id2word = dict(zip(word2id.values(), word2id.keys()))

print("Tokens count:", len(tokens))
print("Unique tokens:", len(set(tokens)))
print("Vocabulary size:", vocabulary_size)
print("Most common tokens:", count[:10])

Tokens count: 17005207
Unique tokens: 253854
Vocabulary size: 47135
Most common tokens: [('UNK', 444176), ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764), ('in', 372201), ('a', 325873), ('to', 316376), ('zero', 264975), ('nine', 250430)]


## Making batches

In [10]:
import collections
import random

data_index = 0
# Generate training batch for the skip-gram model.
def next_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    # get window size (words left and right + current one).
    span = 2 * skip_window + 1
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            buffer.extend(data[0:span])
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    # Backtrack a little bit to avoid skipping words in the end of a batch.
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels

## Defining the Model

#### Embedding
The embedding matrix has a size of the number of words by the number of units in the hidden layer. So, if we have 10,000 words and 300 hidden units, the matrix will have size  10,000×300 . Note that we're using tokenized data for our inputs, usually as integers, where the number of tokens is the number of words in our vocabulary.

#### Parameters

In [11]:
embedding_size =  200  # Number of embedding features

num_sampled = 64  # Number of negative examples to sample.

In [12]:
class MyModel(tf.Module):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # Create the embedding variable (each row represent a word embedding vector).
        self.embedding = tf.Variable(tf.random.normal([vocabulary_size, embedding_size]))
        # Construct the variables for the NCE loss.
        self.nce_weights = tf.Variable(tf.random.normal([vocabulary_size, embedding_size]))
        self.nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

    def __call__(self, x):
        # Lookup the corresponding embedding vectors for each sample in X.
        return tf.nn.embedding_lookup(self.embedding, x)

model = MyModel()

2024-03-06 22:18:24.135714: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Loss function with Negative sampling

For every example we give the network, we train it using the output from the softmax layer. That means for each input, we're making very small changes to millions of weights even though we only have one true example. This makes training the network very inefficient. We can approximate the loss from the softmax layer by only updating a small subset of all the weights at once. We'll update the weights for the correct label, but only a small number of incorrect labels. This is called ["negative sampling"](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf). 
We will use tensorflow function [`tf.nn.nce_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/nce_loss) to do this. Compare this with [`tf.nn.sampled_softmax_loss`](https://www.tensorflow.org/api_docs/python/tf/nn/sampled_softmax_loss)


In [13]:
def nce_loss_function(x_embed, y, nce_weights_,nce_biases_):
    #with tf.device('/cpu:0'):
    # Compute the average NCE loss for the batch.
    y = tf.cast(y, tf.int64)
    loss = tf.reduce_mean(
        tf.nn.nce_loss(weights=nce_weights_,
                    biases=nce_biases_,
                    labels=y,
                    inputs=x_embed,
                    num_sampled=num_sampled,
                    num_classes=vocabulary_size))
    return loss

## Evaluation Function

Compute the cosine similarity between input data embedding and every embedding vectors

In [14]:
def evaluate(model,x_embed):
    #with tf.device('/cpu:0'):
    # Compute the cosine similarity between input data embedding and every embedding vectors
    x_embed = tf.cast(x_embed, tf.float32)
    x_embed_norm = x_embed / tf.sqrt(tf.reduce_sum(tf.square(x_embed)))
    embedding_norm = model.embedding / \
        tf.sqrt(tf.reduce_sum(tf.square(model.embedding),
                                1, keepdims=True), tf.float32)
    cosine_sim_op = tf.matmul(
        x_embed_norm, embedding_norm, transpose_b=True)
    return cosine_sim_op

## Training function

In [15]:
def train(model, x, y, optimizer):
    #with tf.device('/cpu:0'):
    # Wrap computation inside a GradientTape for automatic differentiation.
    with tf.GradientTape() as t:
        #emb = get_embedding(x)
        emb = model(x)
        loss = nce_loss_function(emb, y,model.nce_weights,model.nce_biases)

    # Compute gradients.
    gradients = t.gradient(loss, [model.embedding, model.nce_weights, model.nce_biases])

    # Update W and b following gradients.
    optimizer.apply_gradients(zip(gradients, [model.embedding, model.nce_weights, model.nce_biases]))

## Training

Every display_step batches it reports the training loss. Every eval_step batches, it'll print out the validation words.

In [16]:
# Words for testing.
eval_words = ['five','of','going','hardware','american','britain']
x_test = np.array([word2id[w] for w in eval_words])
print(x_test)

[  16    2 1344 1794   64  703]


#### Training parameters

In [17]:
num_steps  = 10000 #3000000 #10000
display_step = 1000 # 10000 #1000
eval_step = 2000 # 200000 #2000
batch_size = 128
num_skips = 2  # How many times to reuse an input to generate a label.
skip_window = 3  # How many words to consider left and right.

learning_rate = 0.1

In [18]:
# Define the optimizer.
optimizer = tf.optimizers.SGD(learning_rate)

# Run training for the given number of steps.
for step in range(1, num_steps + 1):
    batch_x, batch_y = next_batch(batch_size, num_skips, skip_window)
    train(model, batch_x, batch_y, optimizer)

    if step % display_step == 0 or step == 1:
        loss = nce_loss_function(model(batch_x), batch_y,model.nce_weights,model.nce_biases)
        print("step: %i, loss: %f" % (step, loss))

    # Evaluation.
    if step % eval_step == 0 or step == 1:
        print("Evaluation...")
        sim = evaluate(model, model(x_test)).numpy()
        for i in range(len(eval_words)):
            top_k = 8  # number of nearest neighbors.
            nearest = (-sim[i, :]).argsort()[1:top_k + 1]
            log_str = '"%s" nearest neighbors:' % eval_words[i]
            for k in range(top_k):
                log_str = '%s %s,' % (log_str, id2word[nearest[k]])
            print(log_str)
print('done')

step: 1, loss: 514.151794
Evaluation...
"five" nearest neighbors: quadrants, specials, bibles, zimmer, perfective, risings, clemency, naturalism,
"of" nearest neighbors: decrypts, titian, dionysius, pardons, humanists, amazonas, floating, acceded,
"going" nearest neighbors: honorable, sorghum, punishable, funk, swearing, don, bolstered, adverb,
"hardware" nearest neighbors: irish, plazas, piloting, instigator, prospects, odour, ghetto, parke,
"american" nearest neighbors: holt, stormed, alloying, redstone, outwardly, vecchio, rhyming, transitional,
"britain" nearest neighbors: like, amstrad, mormonism, protector, cahiers, manchurian, undated, grateful,
step: 1000, loss: 264.601135
step: 2000, loss: 218.911957
Evaluation...
"five" nearest neighbors: two, is, and, in, one, a, six, for,
"of" nearest neighbors: and, the, to, in, a, s, is, one,
"going" nearest neighbors: honorable, funk, swearing, punishable, sorghum, don, renoir, adverb,
"hardware" nearest neighbors: irish, plazas, pilotin

## References


* A really good [conceptual overview](http://mccormickml.com/2016/04/19/word2vec-tutorial-the-skip-gram-model/) of word2vec from Chris McCormick 
* [First word2vec paper](https://arxiv.org/pdf/1301.3781.pdf) from Mikolov et al.
* [NIPS paper](http://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) with improvements for word2vec also from Mikolov et al.
* TensorFlow [word2vec tutorial](https://www.tensorflow.org/tutorials/word2vec)
* [An implementation](https://github.com/sminerport/word2vec-skipgram-tensorflow/blob/master/src/word2vec.py)